In [9]:
import os
import subprocess
import shutil
from datetime import datetime

import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling


# --- USER SETTINGS ----------------------------------------------------------

# Base folders (adjust if needed)
input_dir   = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Input"
output_base = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output"

# Path to your WaTEM/SEDEM executable
exe_path    = r"C:\Users\dplatero\watem-sedem-master\watem_sedem\watem_sedem.exe"

# Name of your master .ini (as it sits in `input_dir`)
ini_name    = "Squaw_Creek.ini"
ini_path    = os.path.join(input_dir, ini_name)

# Path to your initial DEM (GeoTIFF) used for post-processing
dem_path    = os.path.join(input_dir, "Squaw.tif")

# Simulation range (inclusive)
year_start, year_end = 2009, 2020

# ------------------------------------------------------------------------------

for year in range(year_start, year_end + 1):
    # --- 1. Read & edit the .ini as plain text ---
    with open(ini_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for L in lines:
        if L.strip().startswith("date"):
            # 1.1 set start date to Jan 1 of this year
            new_lines.append(f"date = 01/01/{year} 00:00:00\n")
        elif L.strip().startswith("output directory"):
            # 1.2 point to a year-specific output folder
            year_out = os.path.join(output_base, str(year))
            # clean up old run if it exists
            if os.path.exists(year_out):
                shutil.rmtree(year_out)
            os.makedirs(year_out, exist_ok=True)
            new_lines.append(f"output directory = {year_out}\n")
        else:
            new_lines.append(L)

    # write a temporary ini for this run
    temp_ini = os.path.join(input_dir, f"run_{year}.ini")
    with open(temp_ini, 'w') as f:
        f.writelines(new_lines)

    # --- 2. Launch WaTEM/SEDEM ---
    print(f"Starting simulation for {year} → output in {year_out}")
    res = subprocess.run([exe_path, temp_ini], cwd=input_dir)
    if res.returncode != 0:
        print(f"  ** ERROR: model returned code {res.returncode}")
        break
    print(f"Finished simulation for {year}")

     # --- 3. Post-process erosion to compute elevation change ---
    erosion_rst = os.path.join(year_out, "WATEREROS (mm per gridcel).rst")
    delta_tif   = os.path.join(year_out, f"delta_elev_{year}.tif")
    updated_dem = os.path.join(year_out, f"DEM_updated_{year}.tif")

    # 3.2.1 Read DEM
with rasterio.open(dem_path) as dem_src:
    dem           = dem_src.read(1).astype("float32")
    profile       = dem_src.profile
    dem_transform = dem_src.transform
    dem_crs       = dem_src.crs
    dem_shape     = dem_src.shape

# 3.2.2 Read raw erosion (rst)
with rasterio.open(erosion_rst, driver="RST") as ero_src:
    raw = ero_src.read(1).astype("float32")

# 3.2.3 Reproject into the DEM’s grid by telling GDAL:
ero_mm = np.zeros(dem_shape, dtype="float32")
reproject(
    source=raw,
    destination=ero_mm,
    # pretend the source already uses the DEM’s CRS/transform:
    src_transform=dem_transform,
    src_crs=dem_crs,
    dst_transform=dem_transform,
    dst_crs=dem_crs,
    resampling=Resampling.nearest,
    init_dest_nodata=True,            # fill any out-of-bounds with zeros
)

ero_m       = ero_mm / 1000.0
dem_updated = dem - ero_m

# 3.3 Write out Δ-elevation (negative = loss)
profile.update(dtype="float32")
with rasterio.open(delta_tif, "w", **profile) as dst:
    dst.write((-ero_m).astype("float32"), 1)

# 3.4 Write out the updated DEM
with rasterio.open(updated_dem, "w", **profile) as dst:
    dst.write(dem_updated.astype(profile["dtype"]), 1)

# 3.5 Point the next iteration at the updated DEM
dem_path = updated_dem

print("All done!")

Starting simulation for 2009 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2009
Finished simulation for 2009
Starting simulation for 2010 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2010
Finished simulation for 2010
Starting simulation for 2011 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2011
Finished simulation for 2011
Starting simulation for 2012 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2012
Finished simulation for 2012
Starting simulation for 2013 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2013
Finished simulation for 2013
Starting simulation for 2014 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2014
Finished simulation for 2014
Starting simulation for 2015 → output in C:\Users\dplatero\watem-sedem-master\watem_sedem\Squaw_Creek\Output\2015
Finished simulation for 2015